In [1]:
#Importing packages we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #pyplot is used to plot the data
import scipy.optimize as opt #used to numerically optimize
from datetime import time, timedelta, datetime
import scipy.stats as stats
import matplotlib.dates as mdates

In [26]:
def GaussianDensity(t,mu, phi, sigma):
    epsilon = y[t] - mu - phi * y[t-1]
    density = stats.norm.pdf(epsilon, 0, sigma)
    return density

# AR(1) likelihood contribution at time t
def ar1_likelihood_contribution(t, mu, phi, sigma):
    if t == 0:
        return 0  # No contribution for the first element
    eps = y[t] - (mu + phi * y[t-1])
    log_likelihood = -0.5 * np.log(2 * np.pi * sigma**2) - (1/(2 * sigma**2)) * eps**2
    return -log_likelihood  # Negative likelihood for minimization

# GARCH(1,1) likelihood contribution at time t
def GarchDensity(t, sigma2, omega, alpha, beta):
    #print(t, simga2)
    if t == 0:
        sigma2[t] = 0.5 # Initial variance
        return 0, sigma2[t]
    else:
        sigma2[t] = omega + alpha * y[t-1]**2 + beta * sigma2[t-1]
    log_likelihood = -0.5 * (np.log(sigma2[t]) + y[t]**2 / sigma2[t])
    return -log_likelihood  # Negative likelihood for minimization

# Function to calculate total likelihood
def total_likelihood(likelihood_function, params, data):
    if likelihood_function == ar1_likelihood_contribution:
        return sum(likelihood_function(params, data, t) for t in range(len(data)))
    elif likelihood_function == garch_likelihood_contribution:
        sigma2 = np.zeros(len(data))
        return sum(likelihood_function(params, data, sigma2, t) for t in range(len(data)))


In [3]:

# # Initial parameter guesses
# initial_ar1_params = [0, 0, 1]  # [alpha, beta, sigma]
# initial_garch_params = [0.01, 0.1, 0.8]  # [omega, alpha, beta]

# # Minimize the negative total likelihood
# ar1_result = minimize(lambda params: total_likelihood(ar1_likelihood_contribution, params, Y), 
#                       initial_ar1_params, method='L-BFGS-B')
# garch_result = minimize(lambda params: total_likelihood(garch_likelihood_contribution, params, Y), 
#                         initial_garch_params, method='L-BFGS-B')

# print("AR(1) model parameters:", ar1_result.x)
# print("GARCH(1,1) model parameters:", garch_result.x)
# # def GaussianDensity(t,mu, phi, sigma):
# #     return np.exp(-0.5*np.log(np.pi)-0.5*np.log(sigma)-0.5*((y[t] - mu - phi * y[t-1])**2)/sigma)

In [23]:
def likelihood(gamma):
    #parameters
    p00    = gamma[0]
    p11    = gamma[1]
    omega = gamma[2:4];
    alpha = gamma[4:6];
    beta = gamma[6:8]**2;
    T      = len(y)
    # print(mu, phi, sigma2)
    #//transition matrix
    P = np.zeros([2,2])
    P[0]=p00, 1-p11
    P[1]=1-p00, p11

    #//bookkeeping
    xi_10      = np.zeros([2,T+1])
    xi_11      = np.zeros([2,T])
    xi_1T      = np.zeros([2,T])
    sigma2     = np.zeros([2,T])
    lik        = np.zeros(T)

    #//regression:
    A  = np.vstack(((np.identity(2)-P),np.ones([1,2])))
    pi_first = np.linalg.inv(A.T.dot(A)).dot(A.T)
    pi_second=np.vstack((np.zeros([2,1]),np.ones([1,1])))
    pi=pi_first.dot(pi_second)
    xi_10[[0,1],0] = pi.T
    # print(f'Pi: {pi.T}')
    #//forward filter recursion
    eta=np.zeros(2)


    for t in range(T):
        #//state densities
        eta[0]=GarchDensity(t, omega[0], alpha[0], beta[0])
        eta[1]=GarchDensity(t, omega[1], alpha[1], beta[1])
        
        #likelihood
        lik[t] = xi_10[0,t] * eta[0] + xi_10[1,t] * eta[1]
        
        #filtering
        num0=eta[0]*xi_10[0,t]/(eta[0]*xi_10[0,t]+eta[1]*xi_10[1,t])
        num1=eta[1]*xi_10[1,t]/(eta[0]*xi_10[0,t]+eta[1]*xi_10[1,t])

        #prediction
        xi_10[[0,1],t+1] = P.dot(xi_11[[0,1],t])
    print(np.sum(lik))
    return -np.sum(lik) #We wish to minimize the likelihood in our scipy opt function



In [24]:
data=np.genfromtxt('SP_500.csv', delimiter=',',usecols=np.arange(0,4)) #loading in first 4 columns
y = data[15097:, 3:4]*100 # 100 times log-returns of the S&P 500 index. January 4, 2010 - till end
print(y)
print("Now transformed:")
y=y.T[0,:] #unpacking numpy array
print(y)
T = len(y) #length of time series

[[ 1.59160816]
 [ 0.31108326]
 [ 0.05453718]
 ...
 [ 1.27496879]
 [ 0.86677408]
 [-0.25643447]]
Now transformed:
[ 1.59160816  0.31108326  0.05453718 ...  1.27496879  0.86677408
 -0.25643447]


In [27]:
# y = data['pct'].to_numpy()
#sigma2 = np.var(y) #used for initial guesses for sigma2 vals
#mean = np.mean(y)
# initial_garch_params = [0.01, 0.1, 0.8]  # [omega, alpha, beta]
Gamma0  = np.array([0.95,0.95, 0.01, 0.1, 0.2, 0.1, 0.8, 0.7]) #initial guesses
res=opt.minimize(likelihood, Gamma0, method='L-BFGS-B',bounds=((0.001,0.9999),(0.001,0.9999),(0.001,None),(0.001,None),(0.001,0.9999),(0.001,0.9999),(0.01,1),(0.01,1))) #optimizing. We use L-BFGS-B as it allows for bounds and can compute the standard errors (from the inverse hessian) right away
res.x
v_hessian=res.hess_inv.todense() #retrieves the negative inverse hessian matrix (note we have minimized the negative log likelihood function)
se_hessian=np.sqrt(np.diagonal(v_hessian))

ValueError: setting an array element with a sequence.

In [17]:
#result of optimization
Gamma_hat=res.x
se=se_hessian
print('P11='+str(Gamma_hat[0])+', std.errors='+str(se[0]))
print('P22='+str(Gamma_hat[1])+', std.errors='+str(se[1]))
print('mu1='+str(Gamma_hat[2])+', std.errors='+str(se[2]))
print('mu2='+str(Gamma_hat[3])+', std.errors='+str(se[3]))
print('phi1='+str(Gamma_hat[4])+', std.errors='+str(se[4]))
print('phi2='+str(Gamma_hat[5])+', std.errors='+str(se[5]))
print('sigma1='+str(Gamma_hat[6])+', std.errors='+str(se[6]))
print('sigma2='+str(Gamma_hat[7])+', std.errors='+str(se[7]))

P11=0.95, std.errors=1.0
P22=0.95, std.errors=1.0
mu1=0.01, std.errors=1.0
mu2=0.1, std.errors=1.0
phi1=0.2, std.errors=1.0
phi2=0.1, std.errors=1.0
sigma1=0.8, std.errors=1.0
sigma2=0.7, std.errors=1.0


In [ ]:
#Plotting returns and filtered probabilities
gamma=Gamma_hat
#parameters
#parameters
p00    = gamma[0]
p11    = gamma[1]
mu = gamma[2:4];
phi = gamma[4:6];
sigma2 = gamma[6:8]**2;
T      = len(y)
#//transition matrix
P = np.zeros([2,2])
P[0]=p00, 1-p11
P[1]=1-p00, p11
    
#//bookkeeping
xi_10      = np.zeros([2,T+1])
xi_11      = np.zeros([2,T])
xi_1T      = np.zeros([2,T])
lik        = np.zeros(T)

#//regression:
A  = np.vstack(((np.identity(2)-P),np.ones([1,2])))
pi_first = np.linalg.inv(A.T.dot(A)).dot(A.T)
pi_second=np.vstack((np.zeros([2,1]),np.ones([1,1])))
pi=pi_first.dot(pi_second)
xi_10[[0,1],0] = pi.T
#//forward filter recursion
eta=np.zeros(2)
for t in range(T):
    #//state densities
    eta[0]=GaussianDensity(t,mu[0], phi[0], sigma2[0])
    eta[1]=GaussianDensity(t,mu[1], phi[1], sigma2[1])
        
    #likelihood
    #print(np.log(xi_10[[0,1],t]))
    lik[t]   = np.log(xi_10[0,t]*eta[0]+xi_10[1,t]*eta[1])
        
    #filtering
    num0=eta[0]*xi_10[0,t]/(eta[0]*xi_10[0,t]+eta[1]*xi_10[1,t])
    num1=eta[1]*xi_10[1,t]/(eta[0]*xi_10[0,t]+eta[1]*xi_10[1,t])
    xi_11[[0,1],t] = num0,num1

    #prediction
    xi_10[[0,1],t+1] = P.dot(xi_11[[0,1],t])
    
    #Backward smoother (not needed for likelihood)
    xi_1T[:,T-1]=xi_11[:,T-1]
    for t in range(T-2,0,-1):
        xi_1T[:,t]=xi_11[:,t]*(P.T.dot(xi_1T[:,t+1]/xi_10[:,t+1]))
        
vol=np.zeros(len(y))
for i in range(T):
    vol[i]=xi_11[[0],i]*sigma2[0]+ (1-xi_11[[0],i])*sigma2[1]



In [ ]:
fig, ax=plt.subplots(2, figsize=(14,7))
#fig.set_figheight=(9)
#fig.set_figwidth=(16)
fig.suptitle('log-return and filtered volatility')
ax[0].plot(y,color='r')
ax[1].plot(np.sqrt(vol))

#Setting titles
ax[0].title.set_text('Log-return, $x_t$')
ax[1].title.set_text('Filtered volatility, $E[\sigma_t|x_t,x_{t-1},...,x_1]$')


In [ ]:
#Predicted state probability, Filtered state probability and smoothed state probability
fig, ax=plt.subplots(3, figsize=(16,9))
#fig.tight_layout() 

#Adjusting size between subplots
fig.subplots_adjust(left=None, bottom=0.025, right=None, top=None, wspace=None, hspace=None)
#default
#left  = 0.125  # the left side of the subplots of the figure
#right = 0.9    # the right side of the subplots of the figure
#bottom = 0.1   # the bottom of the subplots of the figure
#top = 0.9      # the top of the subplots of the figure
#wspace = 0.2   # the amount of width reserved for blank space between subplots
#hspace = 0.2   # the amount of height reserved for white space between subplots


ax[0].plot(1-xi_10[0,:])
ax[1].plot(1-xi_11[0,:])
ax[2].plot(1-xi_1T[0,:])

#Setting limits on x axis
ax[0].set_xlim(0, T)
ax[1].set_xlim(0, T)
ax[2].set_xlim(0, T)

#Setting titles
ax[0].title.set_text('Predicted state probability, $P(s_t=1|x_{t-1},x_{t-2},...,x_{1})$')
ax[1].title.set_text('Filtered state probability, $P(s_t=1|x_{t},x_{t-1},...,x_{1})$')
ax[2].title.set_text('Smoothed state probability, $P(s_t=1|x_{T},x_{T-1},...,x_{1})$')

#Setting lines at 0 and 1
ax[0].axhline(0,color='black', linestyle="--")
ax[0].axhline(1,color='black', linestyle="--")

ax[1].axhline(0,color='black', linestyle="--")
ax[1].axhline(1,color='black', linestyle="--")

ax[2].axhline(0,color='black', linestyle="--")
ax[2].axhline(1,color='black', linestyle="--")
